In [2]:
import numpy as np
import theano
import theano.tensor as tt
import pymc3 as pm

import matplotlib.pyplot as plt




In [3]:
def plain_SIR_model(beta,gamma,N,S0,I0,R0,l=150):
    """Simple classical SIR-Model with time-constant beta,gamma"""
    def next_day(St,It,Rt,oS,oI,oR,eff,beta,gamma,N):
        eff = beta*St/N
        bSIoN = eff * It # -beta * (S*I)/N
        gI = gamma*It
        # Calculate differences
        dS = -bSIoN
        dI = bSIoN - gI
        dR = gI
        # Update values
        nxt_St = St + dS
        nxt_It = It + dI
        nxt_Rt = Rt + dR
        # Pass on to next step
        return nxt_St,nxt_It,nxt_Rt, dS,dI,dR, eff
        
    # Initial internal state
    zero = tt.cast(0.,dtype='float64')
    initial_state = [S0,I0,R0,zero,zero,zero,zero]
    # Exceute the 'Loop'
    state,_ = theano.scan(fn=next_day,
                            n_steps = l,
                            #        sequences=[],
                            outputs_info=initial_state,
                            non_sequences=[beta,gamma,N])
    return state